In [48]:
from wayne_utils import load_data, save_data
import ast
import os
import re
from tqdm import tqdm
os.chdir( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/exec_post" )
from nebula_execute import DB_execute
os.chdir( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/align_db" )
from utils import aligned_or_not, obj_type_diff, get_all_node_name, statistic_obj_len
from copy import deepcopy


## 加载数据并初步评估: mix_intention_list， intention_labels_list

In [57]:
tests = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/test_data_shuffle.jsonl", "jsonl")
trains = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/train_data_shuffle.jsonl", "jsonl")
datasets_schema = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/edge_vertex.json", "json")
test_nl_list = [ tests[i]['prompt'].strip() for i in range(len(tests))]
train_nl_list = [ trains[i]['prompt'] for i in range(len(trains))]
test_gql_list = [ tests[i]["content"] for i in range(len(tests))]       # 1254  # 去重后511
train_gql_list = [ trains[i]["content"] for i in range(len(trains))]    # 3862  # 822

names_space = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/space_name.json", "json")

EA_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intention_aligned_EA_list.json", "json")

In [58]:
intention_temp = {
    "查询对象":[],
    "查询约束":[],
    "返回格式":{
        "distinct": None,
        "aggregation": [],
        "order by": None,
        "limit": None,
        "skip": None
    }
}
'''初始意图
intention_predicts_str_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/prompt_ft/rec_predict_list_N3_intention_rec_Chinese-Mistral-7B-Instruct-v0.1-1.5epoch.pickle", "pickle")
#intention_predicts_str_list = [intention_predicts_raw[i][0].strip() for i in range(len(intention_predicts_raw))]
intention_predicts_str_list[298] = """{'查询对象': [{'category': 'edge', 'label': ['serve'], 'start_node': {'label': []}, 'end_node': {'label': []}}], '查询约束': ["(ID(v) == 'Paul Gasol')"], '返回格式': {'distinct': False, 'aggregation': [], 'order by': None, 'limit': None, 'skip': None}}"""
intention_predicts_str_list[309] = "{'查询对象': [{'category': 'path', 'start_node_id_list': ['Tim Duncan'], 'edge_type_list': ['like'], 'length': None, 'have_direction': True}, {'category': 'path', 'start_node_id_list': ['Tim Duncan'], 'edge_type_list': ['serve'], 'length': '2 steps', 'have_direction': True}], '查询约束': [], '返回格式': {'distinct': False, 'aggregation': [], 'order by': None, 'limit': None, 'skip': None}}"
intention_predicts_str_list[501] = "{'查询对象': [{'category': 'edge', 'label': ['accompany_with'], 'start_node': {'label': ['disease'], 'name': '高血压'}, 'end_node': {'label': ['disease'], 'name': '待查询'}}], '查询约束': [], '返回格式': {'distinct': False, 'aggregation': [], 'order by': None, 'limit': None, 'skip': None}}"
intention_predicts_str_list[774] = "{'查询对象': [{'category': 'edge', 'label': ['accompany_with'], 'start_node': {'label': ['disease'], 'name': '高血压'}, 'end_node': {'label': ['disease'], 'name': '待查询'}}], '查询约束': [], '返回格式': {'distinct': False, 'aggregation': [], 'order by': None, 'limit': None, 'skip': None}}"
intention_predicts_str_list[957] = "{'查询对象': [{'category': 'edge', 'label': ['accompany_with'], 'start_node': {'label': ['disease'], 'name': '高血压'}, 'end_node': {'label': ['disease'], 'name': '待查询'}}], '查询约束': [], '返回格式': {'distinct': False, 'aggregation': [], 'order by': None, 'limit': None, 'skip': None}}"

no_ok_list = []
for i in range(len(intention_predicts_str_list)):
    try:        # 1244 / 1254
        if intention_predicts_str_list[i] == "写数据库":
            # [19,125,135,171,382,422,592,682,785,1219]
            #print(i)
            intention_predicts_list.append(intention_temp)
        else:
            intention_predicts_list.append( ast.literal_eval(intention_predicts_str_list[i]) )
    except:
        intention_predicts_list.append( intention_predicts_str_list[i])
        no_ok_list.append( i )

'''
intention_labels_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intentions_label_test.json", "json")
intention_predicts_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intention_aligned_2.json", "json")


for i in range(len(intention_labels_list)):
    if isinstance( intention_labels_list[i], str):
        intention_labels_list[i] = intention_temp

In [59]:
"""
初始意图：
总数1254，完全一致数量469
查询对象一致数量565，
查询约束一致数量928，
返回格式一致数量1112、

第二次对齐后的意图
总数1254，完全一致数量530
查询对象一致数量632，
查询约束一致数量931，
返回格式一致数量1112，
"""
first_aligned_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intention_aligned.json", "json")
not_aligned_list, related_objects, restrict, return_format = aligned_or_not( intention_predicts_list,intention_labels_list )

总数1254，完全一致数量530
查询对象一致数量632，
查询约束一致数量931，
返回格式一致数量1112，


In [60]:
intention_not_aligned = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intention_not_aligned.json", "json")
mix_intention_list = []
for i in range(len(intention_predicts_list)):
    if i not in not_aligned_list:
        mix_intention_list.append( intention_predicts_list[i])
    else:
        temp = {}
        temp["查询对象"] = intention_predicts_list[i]["查询对象"] if i not in related_objects else intention_not_aligned[i]["查询对象"]
        temp["查询约束"] = intention_predicts_list[i]["查询约束"] if i not in restrict else intention_not_aligned[i]["查询约束"]
        temp["返回格式"] = intention_predicts_list[i]["返回格式"] if i not in return_format else intention_not_aligned[i]["返回格式"]
        mix_intention_list.append( temp )

not_aligned_list, related_objects, restrict, return_format = aligned_or_not( mix_intention_list,intention_labels_list )

总数1254，完全一致数量530
查询对象一致数量632，
查询约束一致数量931，
返回格式一致数量1112，


## Faiss引擎构建

In [9]:
space_edge_list = {
    "nba": ['teammate', 'serve', 'like'],
    "potter": ["belong_to", "kindred", "learn_from"],
    "disease":[
        'accompany_with',
        'cure_department',
        'cure_way',
        'recommend_eat',
        'has_symptom',
        'need_check',
        'no_eat',
        'recommend_drug'
    ]
}

In [10]:
label_property_dict = {
    "potter": {                 # 围绕角色询问，角色的属性、关系
        "node":{
            'characters':['name', 'born', 'dead', 'blood', 'marital', 'species', 'gender', 'height', 'hair', 'eye'],     # 670，角色id, 名称、生日、死亡日期、血型、配偶、物种、性别、高度、发型色、眼睛色
            'college': ['name'],        # 学院：8, id, name
            'group':['name']            # 家族：261
        },
        "edge":{
            'belong_to':[],             # 隶属于：959，character - college/group
            'kindred':['rel_type'],               # 亲属关系及其类型：1924，character - character
            'learn_from':[]             # 在哪里学习：140，character - group
        }
    },
    "nba": {            # 围绕球员提问，
        "node": {
            'player':['name', 'age'],               # 球员：52，名、年龄
            'team':['name'],                        # 球队名：30
            'bachelor':['name','speciality']        # 学士学位：1，专业
        },
        "edge":{
            'like':['likeness'],                    # 喜欢程度：82，球员-球员
            'serve':['start_year', 'end_year'],     # 服役：152,球员 - 球队，起止年
            'teammate':['start_year', 'end_year']   # 队友关系：10，球员-球员，起止年
        }
    },
    "disease": {                        # 围绕疾病询问相关问题
        "node":{
            'drug':['name'],            # 药物，3829
            'disease':['name', 'describe', 'prevent', 'cause', 'easy_get', 'cure_lasttime', 'symptom', 'cured_prob', 'get_prob'],   # 8809 疾病，描述、预防、原因、易感人群、治疗期、症状、治愈概率、得病概率
            'food':['name'],            # 食物，4871
            'check':['name'],           # 检查方法，3354
            'department':['name'],      # 门诊部门，55
            'producer':['name'],        # 制造商？实际是药物名，如开封制药苯妥英钠片 17202
            'symptom':['name'],         # 症状 5999
            'cure':['name']             # 治疗方法 545
        },
        "edge":{
            'accompany_with':[],        # 12025 疾病的伴生症状  disease - disease, symptom
            'cure_department':[],       # 8807  疾病的治疗门诊  disease - department
            'cure_way':[],              # 21084 疾病的治疗方法  disease - cure
            'recommend_eat':[],         # 40222 疾病推荐的食物  disease - food
            'has_symptom':[],           # 54711 疾病的症状      disease - symptom
            'need_check':[],            # 39419 疾病需检查      disease - check
            'no_eat':[],                # 22240 疾病禁食用      disease - food
            'recommend_drug':[]         # 14648 疾病常用药物    disease - drug
        }
    }
}
name_dict = {}

total_path = "/home/jiangpeiwen2/jiangpeiwen2/projects/N3_db_construction/dataset"
for space_name in ["disease", "nba", "potter"]:
    name_dict[space_name] = {}
    dir_name = space_name if space_name in ["disease", "nba"] else "harrypotter"
    file_path = os.path.join( total_path, f"{dir_name}/data")
    for entity in label_property_dict[space_name]["node"].keys():
        
        lists = load_data( os.path.join( file_path, f"{entity}.csv"), "csv")
        if entity == "characters":
            entity = "character"
        key_list = lists.keys()
        _name = "name"
        if "name" not in key_list:
            for item in key_list:
                if "name" in item:
                    _name = item
                    break
    
        try:
            
            name_dict[space_name][entity] = list( lists[_name])
        except:
            raise Exception( os.path.join( file_path, f"{entity}.csv") )

In [1]:
from embedding_retrieving import Embedding
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/n3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# 检查并导入 FAISS 库（根据系统配置选择是否使用 AVX2 优化）
import os
from enum import Enum
import numpy as np


from collections import OrderedDict

"""
Two levels of retrieval:
One way is to calculate the score by editing the distance
Another way is to calculate scores based on the similarity of embedding vectors
"""
def dependable_faiss_import(no_avx2=None):
    """
    检查并导入 FAISS 库（根据系统配置选择是否使用 AVX2 优化）。
    如果 no_avx2 为真，或环境变量 FAISS_NO_AVX2 被设置，则加载非 AVX2 版本的 FAISS。
    如果导入失败，则抛出提示用户安装 FAISS 的异常。

    Args:
        no_avx2: Load Faiss strictly with no AVX2 optimization
            so that the vector store is portable and compatible with other devices.
    """
    if no_avx2 is None and "FAISS_NO_AVX2" in os.environ:
        no_avx2 = bool(os.getenv("FAISS_NO_AVX2"))

    try:
        if no_avx2:
            from faiss import swigfaiss as faiss
        else:
            import faiss
    except ImportError:
        raise ImportError(
            "Could not import the Faiss Python package. "
            "Please install it with `pip install faiss-gpu` (for CUDA supported GPU) "
            "or `pip install faiss-cpu` (depending on Python version)."
        )
    return faiss

In [12]:
# 构建、维护和使用 FAISS 向量检索索引。
class FAISS_R3:
    def __init__(self, embedding_list, string_list, topk=1):
        """
        embeddings: 待检索的嵌入向量（一个嵌入向量的列表）。
        doc_store: 节点的元信息（如 CSV 数据），包含其他与嵌入关联的文档信息。
        """
        self.faiss = dependable_faiss_import()
        self.distance_strategy = "EUCLIDEAN_DISTANCE"                          # 根据 DISTANCE_STRATEGY（config中配置为欧氏距离EUCLIDEAN_DISTANCE），初始化 FAISS 索引
        self.embedding_list = embedding_list
        self.string_list = string_list
        self.index = None
        self.normalize_L2 = False
        self.top_k = topk
        self.key_list = string_list
        self.index = self.faiss.IndexFlatL2(len(embedding_list[0]))


    def add_all(self):
        """将所有的嵌入向量添加到 FAISS 索引中。
        支持向量的 L2 正则化（归一化）
        将 embeddings 转换为 float32 类型以兼容 FAISS。"""
        vector = np.array(self.embedding_list, dtype=np.float32)
        if self.normalize_L2:
            self.faiss.normalize_L2(vector)
        self.index.add(vector)

    def search(self, query):
        """基于向量相似性，检索最接近的文档。
        (1)使用 Embedding().embedding_query(query) 将查询转为嵌入向量
        (2)调用 FAISS 索引的 search 方法找到 top_k 个最相似的嵌入。"""
        top_k = self.top_k
        query_emb = Embedding().embedding_query(query)
        query_emb = np.array(query_emb, dtype=np.float32)
        distance, idx = self.index.search(query_emb.reshape(1, -1), k=top_k)
        topk_string = [self.key_list[i] for i in idx[0]]
        return topk_string[0], distance[0][0]
    
    def search_string(self, query):
        """基于向量相似性，检索最接近的文档。
        (1)使用 Embedding().embedding_query(query) 将查询转为嵌入向量
        (2)调用 FAISS 索引的 search 方法找到 top_k 个最相似的嵌入。"""
        if query in self.string_list:
            return query
        else:
            # 完全匹配
            for i in range(len(self.string_list)):
                string = self.string_list[i]
                if string.lower() == query.lower():
                    return string
            # 被包含匹配
            for i in range(len(self.string_list)):
                string = self.string_list[i]
                if len(query ) > 2 and query.lower() in string.lower():
                    return string
            # 包含匹配
            for i in range(len(self.string_list)):
                string = self.string_list[i]
                if string.lower() in query.lower():
                    return string
        return "未找到"

In [13]:
class Db_instance_retriever():
    def __init__( self, name_dict ):
        self.name_dict = name_dict
        self.embedder = Embedding()
        self.embedding_name_dict = {}
        self.embedding_dict()

    def embedding_dict( self ):
        print( "Embedding all query dict")
        for space_name in self.name_dict.keys():        # nba, potter, disease
            self.embedding_name_dict[space_name] = {}
            for label in self.name_dict[space_name].keys():
                print( f"Embedding {space_name}---{label}")
                string_list = self.name_dict[space_name][label]
                embedding_list = self.embedder.embedding(string_list )
                self.embedding_name_dict[space_name][ label ] = FAISS_R3( embedding_list, string_list)
                self.embedding_name_dict[space_name][ label ].add_all()

    def align_string( self, query, space_name, label = None):
        if label != None:
            sub_faiss = self.embedding_name_dict[ space_name ][label]
            string = sub_faiss.search_string(query)
            if string == "未找到":
                print( f"字符串匹配未找到，进行embedding检索")
                ans, distance = sub_faiss.search(query)
                return ans
            else:
                return string
        else:
            space_dict = self.embedding_name_dict[ space_name ]
            # 先遍历字符串检索
            for label in space_dict.keys():
                sub_faiss = space_dict[label]
                string = sub_faiss.search_string(query)
                if string != "未找到":
                    return string
            # 再相似度检索
            max_dis, max_dis_string = 0, None
            for label in space_dict.keys():
                sub_faiss = space_dict[label]
                ans, distance = sub_faiss.search(query)
                if distance > max_dis:
                    max_dis = distance
                    max_dis_string = ans
            return max_dis_string
            

In [14]:
r3_retriever = Db_instance_retriever( name_dict )

Embedding all query dict
Embedding disease---drug
An unexpected error occurred.


Embedding: 100%|██████████| 383/383 [00:06<00:00, 62.16it/s]


Embedding disease---disease
An unexpected error occurred.


Embedding: 100%|██████████| 881/881 [00:11<00:00, 75.67it/s]


Embedding disease---food
An unexpected error occurred.


...

Embedding: 100%|██████████| 67/67 [00:00<00:00, 77.43it/s]


Embedding potter---college
An unexpected error occurred.


Embedding: 100%|██████████| 1/1 [00:00<00:00, 84.31it/s]


Embedding potter---group
An unexpected error occurred.


Embedding: 100%|██████████| 27/27 [00:00<00:00, 75.93it/s]


## 对齐函数构建

In [67]:
intention_predict['查询对象']

[{'category': 'node',
  'label': ['disease'],
  'name': '幼年型慢性关节炎及其伴发的葡萄膜炎',
  'id': '待查询'}]

In [68]:
index = 1009
intention_predict = intention_predicts_list[index]
intention_label = intention_labels_list[index]
query = test_nl_list[index]
align_obj( intention_predict['查询对象'], intention_label['查询对象'], query, index)

[{'category': 'node',
  'label': ['disease'],
  'name': '幼年型慢性关节炎及其伴发的葡萄膜炎',
  'id': '待查询'}]

In [69]:
intention_label

{'查询对象': [{'category': 'node',
   'label': ['disease'],
   'name': '幼年型慢性关节炎及其伴发的葡萄膜炎',
   'get_prob': '待查询'}],
 '查询约束': [],
 '返回格式': {'distinct': False,
  'aggregation': [],
  'order by': None,
  'limit': None,
  'skip': None}}

In [32]:
def extract_quoted_strings(s):
    """
    提取字符串中单引号或双引号中的子字符串。

    :param s: 输入字符串
    :return: 包含子字符串的列表
    """
    # 使用正则表达式匹配单引号或双引号中的内容
    pattern = r"(['\"])(.*?)(\1)"
    matches = re.findall(pattern, s)
    # 返回匹配组中的实际内容部分
    return [match[1] for match in matches]

In [54]:
def align_obj( intention_predict, intention_label, query, index):
    """总数1254，582个obj一致，672个obj不一致{1: 592, 2: 73, 0: 3, 3: 4}
    727个不一致，其中108长度不一致；长度一致的619中长度{1: 607, 2: 12}，273个type不一致(都是长度为1)，所以长度为1且type一致的有279
    """
    # 获取该query对应的space
    obj_predict = intention_predict['查询对象']
    obj_label = intention_label['查询对象']
    space_name = names_space[index]
    # 返回的列表
    ret_obj_list = []
    
    for i in range(len(obj_predict)):
        """
        "category": "edge",
        "label": ["kindred"],
        "rel_type": intention['restrict'][0].split("==")[1].strip(),
        或者
        
        """
        # 处理path edge划分上的模糊
        if i == 0 and len(obj_label) == 1 and len(obj_predict) == 1 and obj_predict[0]["category"] == "edge" and obj_label[0]["category"] == "path":
            if 'start_node_id_list' in obj_label[0]:
                new_obj = trans_edge_to_path( obj_predict[0] )
                if len(intention_predict["查询约束"]) == 1 and "==" in intention_predict["查询约束"][0] and "id" in intention_predict["查询约束"][0]:
                    name_ = extract_quoted_strings(intention_predict["查询约束"][0])
                    intention_predict["查询约束"] = []
                    new_obj['start_node_id_list'].extend( name_)
            else:
                new_obj = deepcopy( obj_predict[i] )
                new_obj["category"] = "path"
        else:
            new_obj = deepcopy( obj_predict[i] )
        # 获取type
        _type = new_obj["category"]
        if _type not in ['edge', 'path', 'node', 'sub_graph']:
            continue
        # 获取label
        if 'label' in new_obj and set(new_obj['label']) == set(space_edge_list[space_name]):
            new_obj["label"] = []
        if "label" in new_obj:
            _label = new_obj["label"]
            if isinstance( _label, str):
                label = _label
            elif isinstance( _label, list) and len(_label)==1:
                label = _label[0]
            else:
                label = None
        else:
            label = None
        
        # 处理sex属性
        if "sex" in new_obj:
            value = new_obj["sex" ]
            del new_obj["sex"]
            new_obj["gender" ] = value
        # 处理有空
        if 'cured_prob ' in new_obj:
            value = new_obj['cured_prob ' ]
            del new_obj['cured_prob ']
            new_obj['cured_prob' ] = value
        # 处理edge没有方向
        if _type == "edge" and "have_direction" not in new_obj:
            if len(obj_label) > i:
                if "have_direction" in obj_label[i]:
                    new_obj["have_direction"] = True
        
        for key in new_obj.keys():
            if key not in ["category", "label"]:
                value = new_obj[key]
                # 处理edge_type_list特殊情况
                if key == 'edge_type_list':
                    if not isinstance( value, list):
                        new_obj[key] = None
                    else:
                        # 罗列出全部关系则不指名
                        if set(new_obj[key]) == set(space_edge_list[space_name]):
                            new_obj[key] = None
                # 处理对象中的起点和终点字典
                elif isinstance( value, dict):
                    if key in ["start_node", "end_node"]:
                        temp_label = None
                        # 检查start_node点的合规情况
                        if "label" in value:
                            if len( value["label"] ) == 1:
                                temp_label = value["label"][0]
                            elif len( value["label"] ) == 0:
                                temp_label = None
                            else:
                                raise Exception( f"{index}: 错误的start_node标签长度：{value}")
                        else:
                            raise Exception( f"{index}: 不存在start_node的标签{intention_predict}")
                        # 遍历对齐
                        for sub_key in value.keys():
                            sub_value = value[sub_key]
                            if sub_key != "label":
                                if sub_value!="待查询":
                                    if temp_label != None and temp_label not in str(label_property_dict[space_name]):
                                        temp_label = None
                                    new_value = r3_retriever.align_string( sub_value, space_name, temp_label)
                                    new_obj[key][sub_key] = new_value
                                else:
                                    new_obj[key][sub_key] = "待查询"
                    else:
                        raise Exception( f"{index}: 对象的值为字典但键不在'start_node,end_node'中{intention_predict}")
                # 处理对象中id list
                elif isinstance( value, list):
                    if key in ["start_node_id_list", "end_node_id_list", "node_id_list"]:
                        new_list = []
                        for id_name in new_obj[key]:
                            if label != None and label not in str(label_property_dict[space_name]):
                                label = None
                            try:
                                new_list.append( r3_retriever.align_string( id_name, space_name, label) )
                            except:
                                raise Exception(obj_predict )
                        new_obj[key] = new_list
                    else:
                        pass
                # 处理直接的字符属性值
                elif isinstance( value, str):
                    if key in ["name", "id"] and value != "待查询":  # 跳过不必要对齐的键值对
                        if label != None and label not in str(label_property_dict[space_name]):
                            label = None
                        new_obj[key] = r3_retriever.align_string( value, space_name, label)
                # 跳过其他的列表
                else:
                    pass
            # 跳过category", "label"键值
            else:
                pass
        ret_obj_list.append( new_obj )
    return ret_obj_list


In [34]:
def align_restrict( res_list, index ):
    space_name = names_space[index]
    name_list = []
    for i in range(len(res_list)):
        name_list.extend( extract_quoted_strings(res_list[i]) )

    new_name_map = {}
    for name in name_list:
        new_name = r3_retriever.align_string( name, space_name, None)
        new_name_map[name] = new_name
    new_res = []
    for res in res_list:
        for name in new_name_map:
            res = res.replace( name, new_name_map[name])
        new_res.append( res )
    return new_res

In [35]:
def align_return_format( format_dict ):
    temp = {
        'distinct': False,
        'aggregation': [],
        'order by': None,
        'limit': None,
        'skip': None
    }
    temp["distinct"] = format_dict["distinct"] if format_dict["distinct"] != None else False
    temp["order by"] = format_dict["order by"] if (isinstance( format_dict["order by"], str) or format_dict["order by"] == None) else None
    temp["limit"] = format_dict["limit"] if (isinstance( format_dict["limit"], str) or format_dict["limit"] == None) else None
    temp["skip"] = format_dict["skip"] if (isinstance( format_dict["skip"], str) or format_dict["skip"] == None) else None
    new_agg = []
    for item in format_dict["aggregation"]:
        if item.lower().strip() in ["min", "max", "sum", "collect", "count", "id", "avg", "label", "type", "concat", "relationships", "properties"]:
            new_agg.append( item.lower().strip() )
    temp["aggregation"] = new_agg
    return temp


In [42]:
def align_intention_list( intention_predicts_list, intention_labels_list, test_nl_list, not_aligned_list, related_objects, restrict, return_format):
    aligned_list = []
    if len( intention_predicts_list ) != len( intention_labels_list ):
        raise Exception( f"长度不一致 { len(intention_predicts_list), len( intention_labels_list)} ")
    for i in tqdm( range( len(intention_predicts_list)), desc = f"Aligning"):
        intention_predict = intention_predicts_list[i]
        query = test_nl_list[i]
        # 已经完全对齐，直接append
        if i not in not_aligned_list:
            aligned_list.append( intention_predict )
        else:
            intention_label = intention_labels_list[i]
            single_temp = {
                "查询对象": None,
                "查询约束": None,
                "返回格式": None
            }
            if i not in related_objects:
                single_temp[ "查询对象" ] = intention_predict[ "查询对象" ]
            else:
                single_temp[ "查询对象" ] = align_obj( intention_predict, intention_label, query, i)

            if i not in restrict:
                single_temp[ "查询约束" ] = intention_predict[ "查询约束" ]
            else:
                single_temp[ "查询约束" ] = align_restrict( intention_predict[ "查询约束" ], i )

            if i not in return_format:
                single_temp[ "返回格式" ] = intention_predict[ "返回格式" ]
            else:
                single_temp[ "返回格式" ] = align_return_format( intention_predict["返回格式"])
            
            single_temp[ "返回格式" ] = intention_predict[ "返回格式" ]
            aligned_list.append( single_temp )
    return aligned_list

In [56]:
def trans_edge_to_path( predict_obj ):
    ret_temp = {
        'category': 'path',
        'start_node_id_list': [],
        'edge_type_list': [],
        'length': None,
        'have_direction': True
    }
    ret_temp['edge_type_list'] = predict_obj['label']
    if "start_node" in predict_obj and 'name' in predict_obj['start_node']:
        ret_temp['start_node_id_list'].append( predict_obj['start_node']['name'] )
    return ret_temp
    

In [61]:
aligned_list = align_intention_list( mix_intention_list, intention_labels_list, test_nl_list, not_aligned_list, related_objects, restrict, return_format)

Aligning:   6%|▌         | 72/1254 [00:04<01:25, 13.90it/s]

字符串匹配未找到，进行embedding检索


Aligning:  17%|█▋        | 210/1254 [00:05<00:19, 52.22it/s]

字符串匹配未找到，进行embedding检索


Aligning:  19%|█▉        | 243/1254 [00:05<00:20, 49.77it/s]

...

Aligning:  87%|████████▋ | 1087/1254 [00:54<00:06, 27.21it/s]

字符串匹配未找到，进行embedding检索


Aligning:  95%|█████████▌| 1195/1254 [00:55<00:01, 51.60it/s]

字符串匹配未找到，进行embedding检索


Aligning: 100%|██████████| 1254/1254 [00:56<00:00, 22.21it/s]


In [64]:
"""
原始：
总数1254，完全一致数量469
查询对象一致数量565，
查询约束一致数量928，
返回格式一致数量1112，

第一次对齐：
总数1254，完全一致数量527
查询对象一致数量632，
查询约束一致数量928，
返回格式一致数量1112，

第二次对齐
总数1254，完全一致数量530
查询对象一致数量632，
查询约束一致数量931，
返回格式一致数量1112，

第三次对齐
总数1254，完全一致数量533
查询对象一致数量640，
查询约束一致数量941，
返回格式一致数量1112，
"""

_, _, _, _ = aligned_or_not( aligned_list,intention_labels_list )

总数1254，完全一致数量533
查询对象一致数量640，
查询约束一致数量941，
返回格式一致数量1112，


In [63]:
save_data( aligned_list, "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intention_aligned_3.json")